In [2]:
import pandas as pd

In [93]:
df = pd.read_csv("/Users/yhk/data/DataCon/DNS/dns_q1/q1_final.csv")

| 字段                     |含义                  |字段                      |含义                     |
| ----------------------- | -------------------- | ----------------------- | ------------------------ |
| frame.len               | 数据长度             | dns.flags.authenticated | 服务器是否为域权威服务器 |
| ip.src                  | 源 ip                | dns.flags.checkdisable  | 非认证数据是否可接收     |
| ip.dst                  | 目的 ip              | dns.flags.rcode         | DNS reply code           |
| udp.srcport             | 源 udp 端口号        | dns.count.queries       | s数据包中 DNS 请求数     |
| udp.dstport             | 目的 udp 端口号      | dns.count.answers       | 数据包中的应答数         |
| eth.src                 | 源 MAC 地址          | dns.count.auth_rr       | 数据包中权威记录数       |
| eth.dst                 | 目的 MAC 地址        | dns.count.add_rr        | 数据包中额外记录数       |
| dns.id                  | DNS Transaction ID   | dns.qry.name            | DNS 请求名               |
| dns.flags.response      | DNS请求/响应标志     | dns.qry.class           | DNS 请求类型             |
| dns.flags.opcode        | DNS opcode           | dns.resp.name           | DNS 响应名               |
| dns.flags.authoritative | 应答是否被服务器认证 | dns.resp.type           | DNS 回复类型             |
| dns.flags.truncated     | 消息是否剪裁         | dns.resp.ttl            | DNS 响应生存时间         |
| dns.flags.recdesired    | 是否递归查询         | dns.resp.z.do           | DNS 是否支持 DNSSEC      |
| dns.flags.reavail       | 服务器是否能递归查询 | frame.time_relative     | frame 的相对时间         |

## 进行数据整体查验

In [40]:
print("总数据量:{}".format(df.shape[0]))
print("源ip数:{}".format(df["ip.src"].drop_duplicates().shape[0]))
print("目的ip数:{}".format(df["ip.dst"].drop_duplicates().shape[0]))

总数据量:10736020
源ip数:144725
目的ip数:146075


### 得到主要的DNS服务器地址

In [77]:
df[df['dns.flags.response']==1].groupby("ip.src").count().sort_values('frame.number',ascending=False).head(5)

,frame.number,frame.time_relative,ip.dst,frame.len,eth.src,eth.dst,udp.srcport,udp.dstport,dns.id,dns.flags.response,...,dns.count.answers,dns.count.auth_rr,dns.count.add_rr,dns.qry.name,dns.qry.type,dns.qry.class,dns.resp.name,dns.resp.type,dns.resp.ttl,dns.resp.z.do
ip.src,,,,,,,,,,,,,,,,,,,,,
45.80.170.1,5219884,5219884,5219884,5219884,5219884,5219884,5202399,5202399,5219884,5219884,...,5218918,5218918,5219884,5219884,5219884,5219884,5218699,5218699,5191418,4806854
182.254.116.116,16227,16227,16227,16227,16227,16227,16227,16227,16227,16227,...,16227,16227,16227,16227,16227,16227,16227,16227,16227,0
119.29.29.29,15719,15719,15719,15719,15719,15719,15719,15719,15719,15719,...,15719,15719,15719,15719,15719,15719,15719,15719,15719,0
187.199.129.12,11868,11868,11868,11868,11868,11868,11868,11868,11868,11868,...,11868,11868,11868,11868,11868,11868,11868,11868,11868,11868
188.141.167.218,11329,11329,11329,11329,11329,11329,11329,11329,11329,11329,...,11329,11329,11329,11329,11329,11329,11329,11329,11319,11329


In [21]:
df.head()

,frame.number,frame.time_relative,ip.src,ip.dst,frame.len,eth.src,eth.dst,udp.srcport,udp.dstport,dns.id,...,dns.count.answers,dns.count.auth_rr,dns.count.add_rr,dns.qry.name,dns.qry.type,dns.qry.class,dns.resp.name,dns.resp.type,dns.resp.ttl,dns.resp.z.do
0,1,0.000000,214.13.212.97,45.80.170.1,88,00:1c:14:0e:92:2b,00:1c:14:86:22:5a,44055.0,53.0,0x0000437f,...,0.0,0.0,1.0,www.94e207.com.cn,1.0,0x00000001,<Root>,41.0,NaN,1.0
1,2,0.000255,45.80.170.1,214.13.212.97,432,00:1c:14:86:22:5a,00:1c:14:0e:92:2b,53.0,44055.0,0x0000437f,...,0.0,5.0,4.0,www.94e207.com.cn,1.0,0x00000001,94e207.com.cn,2.0,172800.0,1.0
2,3,0.001941,255.176.91.32,45.80.170.1,89,00:1c:14:0e:92:2b,00:1c:14:86:22:5a,44935.0,53.0,0x000087a8,...,0.0,0.0,1.0,dns.32477b4.com.cn,28.0,0x00000001,<Root>,41.0,NaN,1.0
3,4,0.002091,45.80.170.1,255.176.91.32,396,00:1c:14:86:22:5a,00:1c:14:0e:92:2b,53.0,44935.0,0x000087a8,...,0.0,4.0,3.0,dns.32477b4.com.cn,28.0,0x00000001,32477b4.com.cn,2.0,172800.0,1.0
4,5,0.011860,214.180.95.136,45.80.170.1,87,00:1c:14:0e:92:2b,00:1c:14:86:22:5a,48304.0,53.0,0x00001658,...,0.0,0.0,1.0,dns2.f0a1.com.cn,28.0,0x00000001,<Root>,41.0,NaN,1.0


In [11]:
df.columns

Index(['frame.number', 'frame.time_relative', 'ip.src', 'ip.dst', 'frame.len',
       'eth.src', 'eth.dst', 'udp.srcport', 'udp.dstport', 'dns.id',
       'dns.flags.response', 'dns.flags.opcode', 'dns.flags.authoritative',
       'dns.flags.truncated', 'dns.flags.recdesired', 'dns.flags.recavail',
       'dns.flags.authenticated', 'dns.flags.checkdisable', 'dns.flags.rcode',
       'dns.count.queries', 'dns.count.answers', 'dns.count.auth_rr',
       'dns.count.add_rr', 'dns.qry.name', 'dns.qry.type', 'dns.qry.class',
       'dns.resp.name', 'dns.resp.type', 'dns.resp.ttl', 'dns.resp.z.do'],
      dtype='object')

In [28]:
df.groupby(['dns.qry.type']).count()

,frame.number,frame.time_relative,ip.src,ip.dst,frame.len,eth.src,eth.dst,udp.srcport,udp.dstport,dns.id,...,dns.count.queries,dns.count.answers,dns.count.auth_rr,dns.count.add_rr,dns.qry.name,dns.qry.class,dns.resp.name,dns.resp.type,dns.resp.ttl,dns.resp.z.do
dns.qry.type,,,,,,,,,,,,,,,,,,,,,
1.0,5641721,5641721,5641721,5641721,5641721,5641721,5641721,5619079,5619079,5641721,...,5641721,5641721,5641721,5641721,5641721,5641721,5290570,5290570,2810368,4939836
2.0,82653,82653,82653,82653,82653,82653,82653,82623,82623,82653,...,82653,82653,82653,82653,82653,82653,81196,81196,41343,79739
5.0,11938,11938,11938,11938,11938,11938,11938,11936,11936,11938,...,11938,11938,11938,11938,11938,11938,11831,11831,5972,11725
6.0,19637,19637,19637,19637,19637,19637,19637,19611,19611,19637,...,10214,10214,10214,19637,19637,19637,13763,13763,10398,6705
12.0,1172,1172,1172,1172,1172,1172,1172,1166,1166,1172,...,1172,1172,1172,1172,1172,1172,730,730,585,287
13.0,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,1,2
15.0,121066,121066,121066,121066,121066,121066,121066,120352,120352,121066,...,121066,121066,121066,121066,121066,121066,107651,107651,60209,94216
16.0,21226,21226,21226,21226,21226,21226,21226,21180,21180,21226,...,21226,21226,21226,21226,21226,21226,19491,19491,10600,17755
28.0,4559865,4559865,4559865,4559865,4559865,4559865,4559865,4548624,4548624,4559865,...,4559865,4559865,4559865,4559865,4559865,4559865,4484742,4484742,2275660,4409546


## 1. 查验类型为255（ANY）的数据是否为放大攻击

#### 查看请求针对的服务器情况

In [68]:
# 查看请求为ANY类型的请求包的目的ip情况
df[(df['dns.qry.type']==255)&(df['dns.flags.response']==0)].groupby("ip.dst").count()

,frame.number,frame.time_relative,ip.src,frame.len,eth.src,eth.dst,udp.srcport,udp.dstport,dns.id,dns.flags.response,...,dns.count.answers,dns.count.auth_rr,dns.count.add_rr,dns.qry.name,dns.qry.type,dns.qry.class,dns.resp.name,dns.resp.type,dns.resp.ttl,dns.resp.z.do
ip.dst,,,,,,,,,,,,,,,,,,,,,
187.199.129.12,11868,11868,11868,11868,11868,11868,11868,11868,11868,11868,...,11868,11868,11868,11868,11868,11868,11868,11868,0,11868
188.141.167.218,17182,17182,17182,17182,17182,17182,17182,17182,17182,17182,...,17182,17182,17182,17182,17182,17182,17182,17182,0,17182
45.80.170.1,18675,18675,18675,18675,18675,18675,18625,18625,18675,18675,...,18675,18675,18675,18675,18675,18675,18223,18223,0,18223
70.85.232.160,4150,4150,4150,4150,4150,4150,4150,4150,4150,4150,...,4150,4150,4150,4150,4150,4150,4150,4150,0,4150


In [69]:
# 查看收到了请求类型为ANY并且返回包长度大于2000的DNS服务器情况
df[(df['dns.qry.type']==255)&(df['dns.flags.response']==1)&(df['frame.len']>2000)].groupby("ip.src").count()

,frame.number,frame.time_relative,ip.dst,frame.len,eth.src,eth.dst,udp.srcport,udp.dstport,dns.id,dns.flags.response,...,dns.count.answers,dns.count.auth_rr,dns.count.add_rr,dns.qry.name,dns.qry.type,dns.qry.class,dns.resp.name,dns.resp.type,dns.resp.ttl,dns.resp.z.do
ip.src,,,,,,,,,,,,,,,,,,,,,
187.199.129.12,11868,11868,11868,11868,11868,11868,11868,11868,11868,11868,...,11868,11868,11868,11868,11868,11868,11868,11868,11868,11868
188.141.167.218,11319,11319,11319,11319,11319,11319,11319,11319,11319,11319,...,11319,11319,11319,11319,11319,11319,11319,11319,11319,11319
45.80.170.1,4,4,4,4,4,4,0,0,4,4,...,4,4,4,4,4,4,4,4,4,0
70.85.232.160,3523,3523,3523,3523,3523,3523,3523,3523,3523,3523,...,3523,3523,3523,3523,3523,3523,3523,3523,3523,3523


&emsp;&emsp;这里可以看出ip:187.199.129.12、188.141.167.218、70.85.232.160请求类型为ANY的报文绝大部分报文等的返回包都很大，超过了2000，因此基本上可以确定受到了放大攻击。  
&emsp;&emsp;对于45.80.170.1，则很有可能是被尝试使用放大攻击进行攻击，但是没有成功

#### 锁定放大攻击的攻击者

In [74]:
df[(df['dns.qry.type']==255)&(df['dns.flags.response']==1)&(df['frame.len']>2000)].groupby(['ip.src','ip.dst']).count()


,,frame.number,frame.time_relative,frame.len,eth.src,eth.dst,udp.srcport,udp.dstport,dns.id,dns.flags.response,dns.flags.opcode,...,dns.count.answers,dns.count.auth_rr,dns.count.add_rr,dns.qry.name,dns.qry.type,dns.qry.class,dns.resp.name,dns.resp.type,dns.resp.ttl,dns.resp.z.do
ip.src,ip.dst,,,,,,,,,,,,,,,,,,,,,
187.199.129.12,127.130.104.152,11868,11868,11868,11868,11868,11868,11868,11868,11868,11868,...,11868,11868,11868,11868,11868,11868,11868,11868,11868,11868
188.141.167.218,175.222.102.169,11319,11319,11319,11319,11319,11319,11319,11319,11319,11319,...,11319,11319,11319,11319,11319,11319,11319,11319,11319,11319
45.80.170.1,65.120.116.251,4,4,4,4,4,0,0,4,4,4,...,4,4,4,4,4,4,4,4,4,0
70.85.232.160,105.191.150.205,3523,3523,3523,3523,3523,3523,3523,3523,3523,3523,...,3523,3523,3523,3523,3523,3523,3523,3523,3523,3523


In [140]:
df_Aplication_attack = df[((df['ip.src']=='127.130.104.152')|(df['ip.src']=='175.222.102.169')|(df['ip.src']=='105.191.150.205'))&(df['dns.qry.type']==255)&(df['dns.flags.response']==0)]
df_Aplication_attack.shape

(33200, 30)

## 2. 区域传送攻击

检测角度：单个ip地址大量dns服务器进行区域传送请求

In [143]:
df[(df['dns.qry.type']==252)&(df['dns.flags.response']==0)].groupby(["ip.src","ip.dst"]).count().head(15)

frame.number  frame.time_relative  frame.len  \
ip.src         ip.dst                                                         
129.191.74.107 45.80.170.1                1                    1          1   
221.223.19.169 45.80.170.1                2                    2          2   
96.199.230.176 1.172.165.68               1                    1          1   
               1.253.50.142               1                    1          1   
               1.60.62.13                 1                    1          1   
               10.121.102.190             1                    1          1   
               10.171.135.202             1                    1          1   
               10.230.172.68              1                    1          1   
               10.45.121.244              1                    1          1   
               10.64.120.201              1                    1          1   
               100.16.21.248              1                    1          1   
               100.167.60.12              1                    1          1   
               100.5.39.8                 4                    4          4   
               100.73.136.217             1                    1          1   
               101.204.83.47              1                    1          1   

                               eth.src  eth.dst  udp.srcport  udp.dstport  \
ip.src         ip.dst                                                       
129.191.74.107 45.80.170.1           1        1            0            0   
221.223.19.169 45.80.170.1           2        2            0            0   
96.199.230.176 1.172.165.68          1        1            0            0   
               1.253.50.142          1        1            0            0   
               1.60.62.13            1        1            0            0   
               10.121.102.190        1        1            0            0   
               10.171.135.202        1        1            0            0   
               10.230.172.68         1        1            0            0   
               10.45.121.244         1        1            0            0   
               10.64.120.201         1        1            0            0   
               100.16.21.248         1        1            0            0   
               100.167.60.12         1        1            0            0   
               100.5.39.8            4        4            0            0   
               100.73.136.217        1        1            0            0   
               101.204.83.47         1        1            0            0   

                               dns.id  dns.flags.response  dns.flags.opcode  \
ip.src         ip.dst                                                         
129.191.74.107 45.80.170.1          1                   1                 1   
221.223.19.169 45.80.170.1          2                   2                 2   
96.199.230.176 1.172.165.68         1                   1                 1   
               1.253.50.142         1                   1                 1   
               1.60.62.13           1                   1                 1   
               10.121.102.190       1                   1                 1   
               10.171.135.202       1                   1                 1   
               10.230.172.68        1                   1                 1   
               10.45.121.244        1                   1                 1   
               10.64.120.201        1                   1                 1   
               100.16.21.248        1                   1                 1   
               100.167.60.12        1                   1                 1   
               100.5.39.8           4                   4                 4   
               100.73.136.217       1                   1                 1   
               101.204.83.47        1                   1                 1   

                               ..

&emsp;&emsp;可以看到96.199.230.176使用大量的区域传送请求，可以确定它应该是一个区域传送攻击的攻击源

In [137]:
df_zone_tranfer = df[(df['ip.src']=='96.199.230.176')&(df['dns.qry.type']==252)]
df_zone_tranfer.shape

(5292, 30)

## 3. DNSSec域名遍历攻击

检测角度：
    1. 一个ip对Dns服务器大量使用qry_type为43的请求不存在的域名
    2. 请求域名的格式都为一个子网下的域名，存在明显的遍历格式，名字中含有*通配符

In [146]:
df[(df['dns.qry.type']==43)&(df['dns.flags.response']==0)].groupby(['ip.src','ip.dst']).count().sort_values('frame.number',ascending=False).head(5)

,,frame.number,frame.time_relative,frame.len,eth.src,eth.dst,udp.srcport,udp.dstport,dns.id,dns.flags.response,dns.flags.opcode,...,dns.count.answers,dns.count.auth_rr,dns.count.add_rr,dns.qry.name,dns.qry.type,dns.qry.class,dns.resp.name,dns.resp.type,dns.resp.ttl,dns.resp.z.do
ip.src,ip.dst,,,,,,,,,,,,,,,,,,,,,
163.194.40.114,45.80.170.1,172,172,172,172,172,172,172,172,172,172,...,172,172,172,172,172,172,172,172,0,172
6.116.183.244,185.25.160.3,71,71,71,71,71,71,71,71,71,71,...,71,71,71,71,71,71,71,71,0,71
239.211.193.162,45.80.170.1,7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,0,7
24.166.221.121,45.80.170.1,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,0,5
120.227.147.90,45.80.170.1,4,4,4,4,4,3,3,4,4,4,...,4,4,4,4,4,4,4,4,0,4


In [148]:
df[(df['ip.dst']=='163.194.40.114')&(df['dns.flags.response']!=0)&(df['ip.src']=='45.80.170.1')][['dns.qry.name','dns.qry.type','dns.resp.name','dns.resp.type','dns.flags.rcode']].head(15)

,dns.qry.name,dns.qry.type,dns.resp.name,dns.resp.type,dns.flags.rcode
241140,26e58.com.cn,43.0,com.cn,6.0,0.0
649118,cb52.com.cn,43.0,com.cn,6.0,0.0
662025,05df.com.cn,43.0,com.cn,6.0,0.0
689422,addf.com.cn,43.0,com.cn,6.0,0.0
840430,97.com.cn,43.0,com.cn,6.0,0.0
911687,d68.com.cn,43.0,com.cn,6.0,0.0
942787,c34.com.cn,43.0,com.cn,6.0,0.0
955361,9c4.com.cn,43.0,com.cn,6.0,0.0
1330332,f9d853.com.cn,43.0,com.cn,6.0,0.0
1438966,d128.com.cn,43.0,com.cn,6.0,0.0


可以看出163.194.40.114的访问请求基本上都得到了正常的响应，因此不是DNSSEC域名遍历攻击

下面来对185.25.160.3的请求进行查验：

In [147]:
df[(df['ip.dst']=='6.116.183.244')&(df['dns.flags.response']==1)&(df['ip.src']=='185.25.160.3')][['dns.qry.name','dns.qry.type','dns.resp.name','dns.resp.type','dns.flags.rcode']].head(10)


,dns.qry.name,dns.qry.type,dns.resp.name,dns.resp.type,dns.flags.rcode
5300243,e24561.com.cn,6.0,e24561.com.cn,6.0,0.0
5300372,***.e24561.com.cn,43.0,e24561.com.cn,6.0,3.0
5300487,admin*.e24561.com.cn,43.0,e24561.com.cn,6.0,3.0
5300646,administrators*.e24561.com.cn,43.0,e24561.com.cn,6.0,3.0
5300746,air*.e24561.com.cn,43.0,e24561.com.cn,6.0,3.0
5300892,aliyum*.e24561.com.cn,43.0,e24561.com.cn,6.0,3.0
5301026,aliyumn*.e24561.com.cn,43.0,e24561.com.cn,6.0,3.0
5301159,aliyun*.e24561.com.cn,43.0,e24561.com.cn,6.0,3.0
5301253,baidu*.e24561.com.cn,43.0,e24561.com.cn,6.0,3.0
5301376,centent*.e24561.com.cn,43.0,e24561.com.cn,6.0,3.0


从上面的结果可以看出DNS请求基本上全部都是43，并且请求从域名上可以明显看出域名遍历的域名格式，而rcode基本上全部都为3（域名不存在）则可以确认该通信为DNSSec攻击

找出全部请求

In [37]:
df_dnssec_attack = df[(df['ip.src']=='6.116.183.244')&(df['dns.flags.response']==0)&(df['ip.dst']=='185.25.160.3')]
df_dnssec_attack.shape

(72, 30)

## 4.未授权的动态更新

检测角度：
    1. opcode为5，找出大量请求的ip
    2. 进一步筛查，各个源ip的opcode为5的请求出现大量响应码rcode为6(refused)    证明非法，只是在试探是否允许非授权的动态更新

In [43]:
df[(df['dns.flags.opcode']==5)&(df['dns.flags.response']==0)].groupby(['ip.src']).count()

,frame.number,frame.time_relative,ip.dst,frame.len,eth.src,eth.dst,udp.srcport,udp.dstport,dns.id,dns.flags.response,...,dns.count.answers,dns.count.auth_rr,dns.count.add_rr,dns.qry.name,dns.qry.type,dns.qry.class,dns.resp.name,dns.resp.type,dns.resp.ttl,dns.resp.z.do
ip.src,,,,,,,,,,,,,,,,,,,,,
18.100.48.86,4091,4091,4091,4091,4091,4091,4091,4091,4091,4091,...,0,0,4091,4091,4091,4091,4091,4091,4091,0
19.220.251.87,414,414,414,414,414,414,414,414,414,414,...,0,0,414,414,414,414,414,414,414,0
200.152.141.106,15,15,15,15,15,15,15,15,15,15,...,0,0,15,15,15,15,15,15,15,0
237.205.156.233,535,535,535,535,535,535,535,535,535,535,...,0,0,535,535,535,535,535,535,535,0


In [149]:
df[df['ip.src']=='18.100.48.86'].groupby('ip.dst').count().head(15)

,frame.number,frame.time_relative,ip.src,frame.len,eth.src,eth.dst,udp.srcport,udp.dstport,dns.id,dns.flags.response,...,dns.count.answers,dns.count.auth_rr,dns.count.add_rr,dns.qry.name,dns.qry.type,dns.qry.class,dns.resp.name,dns.resp.type,dns.resp.ttl,dns.resp.z.do
ip.dst,,,,,,,,,,,,,,,,,,,,,
1.19.167.130,1,1,1,1,1,1,1,1,1,1,...,0,0,1,1,1,1,1,1,1,0
1.2.5.150,1,1,1,1,1,1,1,1,1,1,...,0,0,1,1,1,1,1,1,1,0
1.213.51.10,1,1,1,1,1,1,1,1,1,1,...,0,0,1,1,1,1,1,1,1,0
1.220.152.154,1,1,1,1,1,1,1,1,1,1,...,0,0,1,1,1,1,1,1,1,0
1.25.159.201,1,1,1,1,1,1,1,1,1,1,...,0,0,1,1,1,1,1,1,1,0
1.30.24.223,2,2,2,2,2,2,2,2,2,2,...,0,0,2,2,2,2,2,2,2,0
1.4.251.114,1,1,1,1,1,1,1,1,1,1,...,0,0,1,1,1,1,1,1,1,0
1.47.215.125,1,1,1,1,1,1,1,1,1,1,...,0,0,1,1,1,1,1,1,1,0
1.64.40.187,1,1,1,1,1,1,1,1,1,1,...,0,0,1,1,1,1,1,1,1,0


可以看出18.100.48.86对大量dns服务器进行更新尝试

In [150]:
df[(df['ip.dst']=='18.100.48.86')&(df['dns.flags.response']==1)][['ip.src','dns.flags.opcode','dns.qry.name','dns.resp.name','dns.flags.rcode']].head(15)


,ip.src,dns.flags.opcode,dns.qry.name,dns.resp.name,dns.flags.rcode
4758757,57.19.255.98,5.0,9c830.com.cn,NaN,5.0
4758759,168.192.215.178,5.0,9f1.com.cn,NaN,5.0
4758769,194.42.124.155,5.0,2a1e1.com.cn,NaN,5.0
4758852,126.101.120.227,5.0,1b8c183.com.cn,NaN,5.0
4758855,149.234.197.75,5.0,75ae.com.cn,NaN,5.0
4758859,14.236.90.182,5.0,6332.com.cn,NaN,5.0
4758861,125.76.88.83,5.0,NaN,NaN,5.0
4758873,227.197.132.39,5.0,9f1.com.cn,NaN,5.0
4758885,194.42.124.155,5.0,61dc3.com.cn,NaN,5.0
4758912,93.154.175.80,5.0,ef1e7.com.cn,NaN,5.0


可以粗略的看出18.100.48.86对各个ip地址作虎的更新请求被大量拒绝，并不是一个正常的更新行为，因此确定为未授权的动态更新

其他三个ip地址同理进行查验，也可以进行确定，最终发现全部的opcode为5的请求全部为未授权的动态更新

In [83]:
df_unauthorized_update = df[(df['dns.flags.opcode']==5)&(df['dns.flags.response']==0)]
df_unauthorized_update.shape

(5055, 30)

## 5.Dos攻击

**检测角度**：rcode为3 recdesired为1，即使用递归查询的方式大量请求不存在域名

In [113]:
df[(df['dns.flags.rcode']==3)&(df['dns.flags.recdesired']==1)].shape

(34538, 30)

In [153]:
df[(df['dns.flags.rcode']==3)&(df['dns.flags.recdesired']==1)&(df['dns.flags.response']==1)].groupby(['ip.dst','ip.src']).count().sort_values('eth.src',ascending=False).head(5)

frame.number  frame.time_relative  frame.len  \
ip.dst         ip.src                                                          
144.202.64.226 182.254.116.116         16212                16212      16212   
               119.29.29.29            15712                15712      15712   
               223.6.6.6                1955                 1955       1955   
82.235.92.57   45.80.170.1               492                  492        492   
6.116.183.244  185.25.160.3               71                   71         71   

                                eth.src  eth.dst  udp.srcport  udp.dstport  \
ip.dst         ip.src                                                        
144.202.64.226 182.254.116.116    16212    16212        16212        16212   
               119.29.29.29       15712    15712        15712        15712   
               223.6.6.6           1955     1955         1955         1955   
82.235.92.57   45.80.170.1          492      492          492          492   
6.116.183.244  185.25.160.3          71       71           71           71   

                                dns.id  dns.flags.response  dns.flags.opcode  \
ip.dst         ip.src                                                          
144.202.64.226 182.254.116.116   16212               16212             16212   
               119.29.29.29      15712               15712             15712   
               223.6.6.6          1955                1955              1955   
82.235.92.57   45.80.170.1         492                 492               492   
6.116.183.244  185.25.160.3         71                  71                71   

                                ...  dns.count.answers  dns.count.auth_rr  \
ip.dst         ip.src           ...                                         
144.202.64.226 182.254.116.116  ...              16212              16212   
               119.29.29.29     ...              15712              15712   
               223.6.6.6        ...               1955               1955   
82.235.92.57   45.80.170.1      ...                492                492   
6.116.183.244  185.25.160.3     ...                 71                 71   

                                dns.count.add_rr  dns.qry.name  dns.qry.type  \
ip.dst         ip.src                                                          
144.202.64.226 182.254.116.116             16212         16212         16212   
               119.29.29.29                15712         15712         15712   
               223.6.6.6                    1955          1955          1955   
82.235.92.57   45.80.170.1                   492           492           492   
6.116.183.244  185.25.160.3                   71            71            71   

                                dns.qry.class  dns.resp.name  dns.resp.type  \
ip.dst         ip.src                                                         
144.202.64.226 182.254.116.116          16212          16212          16212   
               119.29.29.29             15712          15712          15712   
               223.6.6.6                 1955           1955           1955   
82.235.92.57   45.80.170.1                492            492            492   
6.116.183.244  185.25.160.3                71             71             71   

                                dns.resp.ttl  dns.resp.z.do  
ip.dst         ip.src                                        
144.202.64.226 182.254.116.116         16212              0  
               119.29.29.29            15712              0  
               223.6.6.6                1955              0  
82.235.92.57   45.80.170.1               492            492  
6.116.183.244  185.25.160.3               71             71  

[5 rows x 28 columns]

144.202.64.226对三个服务器发大量对不存在域名递归查询的解析请求，因此该ip即为攻击者

查看这种类型的请求站该ip发出总请求的百分比：

In [128]:
df[(df['ip.dst']=='144.202.64.226')&(df['dns.flags.rcode']==3)&(df['dns.flags.recdesired']==1)].shape[0]/df[df['ip.dst']=='144.202.64.226'].shape[0]           

0.9992920980444209

可以看出该ip发出的绝大多数请求都是对不存在域名的递归请求，因此该ip的请求流量即为Ddos攻击流量

In [134]:
df_ddos = df[(df['ip.src']=='144.202.64.226')&(df['dns.flags.response']==0)]
df_ddos.shape

(34194, 30)